In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, least_squares
import os
import pandas as pd
import bb
from multiprocessing import Pool

In [4]:
temp = os.path.abspath("second_cut.csv")
name = pd.read_csv(temp, sep=",")
name = pd.DataFrame(name["Name"])
name = np.array( name["Name"].tolist() )

temp = os.path.abspath("redshifts.csv")
redshifts = pd.read_csv(temp, sep=",")

def compute_bol(slsn):
    #slsn - name of object
    # data в магнитудах
    temp = os.path.abspath('data/' + slsn + ".csv")
    ap_data = pd.read_csv(temp, sep=",")
    z = float( redshifts.loc[redshifts["Name"] == slsn]['z'])

    columns = ['mjd', 'T', 'R']
    BB_parameters = pd.DataFrame(columns=columns)
    BB_parameters["mjd"] = ap_data["mjd"]
    data_size = len(ap_data)
    bb_T, bb_R = [], []
    fun = []
    succ = []
    logL, sigma_logL = [], []
    
    pred_params = np.array([1.4, 0.1])
    for d in range(data_size):
        result = minimize(bb.sum_squared_err, pred_params,
                          args=(ap_data.iloc[d], z),
                          method = 'Newton-CG',
                          jac=bb.band_mag_jac,
                          hess=bb.band_mag_hess
                         )
        bb_T.append(result.x[0] * bb.const[0])
        bb_R.append(result.x[1] * bb.const[1])
        succ.append(result.success)
        fun.append(result.fun)
        pred_params = (result.x[0], result.x[1])
        logL.append(np.log10(bb.L(pred_params)))
        sigma_logL.append(bb.sigma_logL(pred_params, ap_data.iloc[d], z))

    BB_parameters["T"], BB_parameters["R"] = bb_T, bb_R
    BB_parameters["fun"] = fun
    BB_parameters["success"] = succ
    BB_parameters['logL'] = logL
    BB_parameters['sigma_logL'] = sigma_logL

    BB_parameters.to_csv(f'bol_output/data/{slsn}.csv', index=False)

In [ ]:
pool = Pool(processes=2)
pool.map(compute_bol, ['PTF12dam', 'SN2006gy'])
pool.close()
pool.join()

/tmp/ipykernel_3510/1885459514.py:14: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  z = float( redshifts.loc[redshifts["Name"] == slsn]['z'])
/tmp/ipykernel_3510/1885459514.py:14: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  z = float( redshifts.loc[redshifts["Name"] == slsn]['z'])
